## やりたいこと
新型Augmentationモジュール動作テスト1 データセットが存在する場合にロードする

In [1]:
# cording = UTF-8
import os,re,random,copy                    #標準ライブラリ
import scipy,librosa,sklearn,joblib,h5py    #サードパーティライブラリ
import numpy as np

aug_amount = 256    #ファイルごとのAugmentationの回数 2の階乗値
aug_length = 5      #Augmentation後のデータ長(秒) 音声データより長いこと
wav_sr = 22050      #音声ファイルのサンプリングレート 44100か22050

base_dir = "../"
data_dir =os.path.join(base_dir,"data")
ok_dir = os.path.join(data_dir,"OK")
ng_dir = os.path.join(data_dir,"NG")
env_dir = os.path.join(data_dir,"environment")
env_file = "masuho_env.wav"

dataset_file = "dataset2.npz"


#Augmentationのメソッド定義クラス
class Aug_method:
    def __init__(self):
        pass

    #対象フォルダ内のWaveファイルの一覧を取得
    def wav_search(self,dir):
        #初期化
        x = []

        os.chdir(dir)
        for i in os.listdir(dir):
            search_index = re.search('.wav',i)
            if search_index:
                x.append(i)
        
        print("Files to process:\n\
        {}".format(x))
        
        return x

    #オーディオファイルの読み込み モノラル固定
    def load_wav(self,dir,file,rate = 44100):
        #初期化
        x = np.arange(0)

        x,y = librosa.load(
            os.path.join(dir,file),
            sr = rate,
            mono =True
        )
        del y
        return x

    #Augmentation処理1 データの耳をそろえてスタート位置を変更
    #これでファイルサイズサーチが必要なくなる…はず
    def shift_wav(self,wav,rate,length):
        #初期化
        x = np.arange(0)

        shift_val=int(rate*length) - int(len(wav))

        x = np.roll(
            np.concatenate(
                [wav,np.zeros(shift_val)]
            ),
            random.randint(0,shift_val)
        )

        return x

    #Augmentation処理2 ノイズの追加
    def add_noize(self,wav):
        x = np.random.randn(len(wav))*random.uniform(0,0.01)
        return x

    #音声のランダム切り出し(主に環境音データ用)
    def wav_extraction(self,wav,rate,length):
        width = int(rate*length)
        start = random.randint(0,int(len(wav)-width))   #最大でもwidth分は残す

        x = copy.deepcopy(wav[start:start+width])

        return x

    #スペクトログラム取得 スペクトラムスケール・マグニチュード
    def get_spg(self,wav,rate):
        #初期化
        x = np.arange(0)

        freq,time,x = scipy.signal.spectrogram(
            wav,
            fs = rate,
            window = np.hamming(1024),
            nfft = 1024,
            scaling = "spectrum",
            mode = "magnitude"
        )
        del freq,time
        return x

#Augmentationの処理クラス
class Proc_aug(Aug_method):
    def __init__(self):
        pass

    #単一のファイルに対する水増し操作
    def aug_data(self,wav,env_wav,rate,length,aug_amount):
        for i in aug_amount:
            # 1.読み込んだデータを成形しスタート位置をランダムシフト
            wf = super().shift_wav(wav,rate,length)

            # 2.環境音データをランダムで切り出し、wfに貼り付ける
            env = super().wav_extraction(env_wav,rate,length)
            env = env * random.triangular(0.01,10,1)   # 1/100～10、最頻値1をかけてS/Nを振る
            wf = wf + env

            # 3.全体に一様分布のノイズを付与
            wf = super().add_noize(wf)
            
            #スペクトラムを得る
            spg = super().get_spg(wf,rate)

            #ファイルを積み上げる
            try:
                x
            except:
                x = copy.deepcopy(spg)
            else:
                x = np.vstack(x,spg)
            del wf,env,spg
        
        return x,i

    #データセットの作成
    def aug_dataset(self,dir,env_dir,env_file,rate,length,aug_amount):
        #ウェーブリストを読み込む
        wave_list = self.wav_search(dir)
        #カウンター変数のリセット
        counter = 0 
        #ウェーブリストを変数としてforループを組む
        for i in wave_list:
            #load_wavでファイルを読み出し、aug_dataに渡す
            wav = super().load_wav(dir,i)
            env_wav = super().load_wav(env_dir,env_file)
            auged_spg,count = self.aug_data(wav,env_wav,rate,length,aug_amount)
            del wav
            
            #データをスタックし、混ぜる(追加学習を見越して)
            try:
                x
            except:
                x = copy.deepcopy(auged_spg)
            else:
                x = np.vstack(x,auged_spg)

            #カウンターを積み上げる
            counter = counter + count
            print ("Augmentation count = {}".format(count))

        #出来上がったデータを混ぜる
        np.random.shuffle(x)
        #変数を返す前に中間生成物を消去
        del env_wav,auged_spg,count,counter
        #変数を返す
        return x

    #初期学習用データセットの作成とセーブ・ロード
    def make_dataset(self,data_dir,ok_dir,ng_dir,env_dir,env_file,rate,length,aug_amount,dataset_file):
        #OKDirをaug_datasetに渡してOKデータセット作成
        X_ok = copy.deepcopy(
            self.aug_dataset(ok_dir,env_dir,env_file,rate,length,aug_amount)
            )
        y_ok = np.zeros(len(X_ok),dtype = 'bool')   #OKデータをfalse(陰性)と定義
            #NGDirをaug_datasetに渡してNGデータセット作成
        X_ng = copy.deepcopy(
            self.aug_dataset(ng_dir,env_dir,env_file,rate,length,aug_amount)
        )            
        y_ng = np.ones(len(X_ng),dtype = 'bool')    #NGデータをTrue(陽性)と定義
        #両者をスタック
        X_data = np.vstack((X_ok,X_ng))
        y_data = np.append(y_ok,y_ng)
        del X_ok,y_ok,X_ng,y_ng
        
        #保存する
        np.savez_compressed(os.path.join(data_dir,dataset_file),
        X = X_data,y = y_data)
        print("Data set saved! : {}".format(str(dataset_file)))

        return X_data,y_data

    def load_dataset(self,data_dir,dataset_file):
        load_data = np.load(os.path.join(data_dir,dataset_file))
        X_data =load_data['X']
        y_data = load_data['y']
        del load_data
        print("Data loaded!!")

        return X_data,y_data

#メイン処理
Aug = Proc_aug()
if os.path.exists(
    os.path.join(os.path.join(
        data_dir,dataset_file)
        )
    ) == False:
    X_data,y_data = Aug.make_dataset(
        data_dir,ok_dir,ng_dir,env_dir,env_dir,wav_sr,aug_length,aug_amount,dataset_file
        )
else:
    key = ""
    while key == "0" or "1":
        key = input ("Do you want to update the dataset? [ yes:0 / n0:1 ]")
        if key == "0":
            X_data,y_data = Aug.make_dataset(
            data_dir,ok_dir,ng_dir,env_dir,env_dir,wav_sr,aug_length,aug_amount,dataset_file
            )
            break
        elif key == "1":
            X_data,y_data = Aug.load_dataset(data_dir,dataset_file)
            break
del Aug


Data loaded!!


問題なく動作。  
ただし、終了後Key変数が残っていたため削除処理を追加